In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env.public")
load_dotenv("../.env.secret")
import os
from pymongo import MongoClient
import chromadb
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import CharacterTextSplitter
from uuid import uuid4

In [ ]:
MONGO_CLIENT = MongoClient(
    os.environ["MONGO_URL"],
    int(os.environ["MONGO_PORT"]),
)
CHROMA_CLIENT = chromadb.HttpClient(
    os.environ["CHROMA_URL"],
    int(os.environ["CHROMA_PORT"]),
)

In [ ]:
ECOM_URL = "https://cloud.google.com/pricing/"
res = requests.get(ECOM_URL)

In [ ]:
text = BeautifulSoup(res.text).get_text()

In [ ]:
text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=256,
    chunk_overlap=128,
    length_function=len,
    is_separator_regex=False,
)
chunks = [document.page_content for document in text_splitter.create_documents([text])]

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        return [[1., 1., 1.] for _ in range(len(input))]

In [ ]:
# sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
CHROMA_CLIENT.delete_collection("test_collection")
collection = CHROMA_CLIENT.get_or_create_collection("test_collection", embedding_function=MyEmbeddingFunction())

In [ ]:
collection.add(
    documents=chunks,
    ids=[str(uuid4()) for _ in range(len(chunks))],
)

In [ ]:
collection.query(
    query_texts="How much free credits do you have?",
    n_results=3,
)